In [ ]:
from IPython.core.display import HTML, Javascript
with open('style.html', 'r') as file:
     css = file.read()
HTML(css)

In [ ]:
%run Util/00_imports.ipynb
%run Util/01_functions.ipynb

# Spielen gegen die KI

> TODO: Referenzen richtig setzen

## Vom Nutzer zu tätigende Einstellungen
Zur Erstellung eines individuellen Spielfeldes werden Dictionaries verwendet. Diese verwenden jeweils als `key` die Figur, 
die auf dem Spielfeld platziert werden soll. Als `value` nutzen die Einträge jeweils eine Liste, 
die vom Nutzer mit den String-Bezeichnern der Felder gefüllt werden sollen, auf denen die jeweilige Figur steht.
Die Bezeichner

> TODO: Welche Bezeichner sind gemeint? Von Spielfeldern?

folgen hierbei dem Format, welches in Notebook `01_chess_introduction` erklärt wurde.

> TODO: Mögliche Formatierung der Beschreibung der globalen Variablen

In [ ]:
WHITE_POSITIONS = {chess.KING:['e1'],
                   chess.QUEEN:['a6'],
                   chess.ROOK:[],
                   chess.BISHOP:[],
                   chess.KNIGHT:[],
                   chess.PAWN:[]}

BLACK_POSITIONS = {chess.KING:['e7'],
                   chess.QUEEN:[],
                   chess.ROOK:[],
                   chess.BISHOP:[],
                   chess.KNIGHT:[],
                   chess.PAWN:[]}

Weiter muss der Nutzer angeben, welche vorher berechnete Spielsituation er laden möchte.
Hierfür muss der Dateiname in einer globalen Variable (ohne Dateiendung `.chessAI`) angegeben werden.

> TODO: Sagen, dass diese in FILE gespeichert wird. Außerdem angeben wo die bezeichnete Datei liegt.

In [ ]:
FILE = "S_n_seq_queen_07_05"

## Logik für die Interaktion mit der Spielsituation
Im folgenden Abschnitt sollen einige Hilfsfunktionen erklärt werden, welche für die Interaktion zwischen dem Nutzer über ein Jupyter Notebook und 
der Spielsituation benötigt werden.  

Die Funktion `get_occupied_cells()` übersetzt die zuvor erstellten und vom Nutzer veränderten `Position-Dictionaries` in eine Liste, die alle ``values`` der
Dictionaries, also besetzte Felder enthält. Diese Liste wird benötigt, um zu überprüfen, ob die Eingaben des Nutzers korrekte Zellen sind. 

In [ ]:
def get_occupied_cells():
    cells = []
    for values in WHITE_POSITIONS.values():
        for value in values:
            cells.append(value)
    for values in BLACK_POSITIONS.values():
        for value in values:
            cells.append(value)
    return cells

Mit der Information von der vorherigen Funktion übernimmt die Funktion `check_for_correct_cells()` die Überprüfung. 
Hierzu wird jeder Wert in den eingegebenen Feldern in den Dictionaries `WHITE_POSITIONS` und `BLACK_POSITIONS` auf Einhaltung
eines regulären Ausdrucks, welcher auf eine Kombination aus Buchstabe und Zahl überprüft. 
Das Ergebnis wird in Form eines booleschen Werts zurückgegeben.

> TODO: Regulären Ausdruck erklären? Bzw. nicht so offen lassen?

In [ ]:
def check_for_correct_cells():
    cells = get_occupied_cells()
    for cell in cells:
        x = re.search("[a-h][1-8]", cell)
        if x is None:
            print(cell)
            print("Value incorrect!")
            return False
        else:
            pass
    return True

Bevor die Figuren dem Objekt hinzugefügt werden, werden sie in einem Dictionary gesammelt, welches als `Piece-Map` verwendet wird.

Die Funktion `collect_cells(color, pieces)` erstellt aus einem Booleschen-Wert für die Spielerfarbe und einem Dictionary mit
Figuren und Spielfeldern eine Liste mit `Piece` 
Objekten, welche als Index den Index des Spielfeldes verwendet. Diese Liste entspricht dem Format, mit welchem Formationen von der Bibliothek geladen werden können.  

> TODO: Nochmal explizit angeben, dass dieses Dictionary Rückgabewert ist
> TODO: Mit wird aus dem Lesen nicht direkt klar, wie pieces aussieht
> TODO: Wird wirklich eine Liste generiert? Kein Dictionary?

In [ ]:
def collect_cells(color, pieces):
    occupied_cells = {}
    for piece_type, values in pieces:
            piece = chess.Piece(piece_type, color)
            for value in values:
                square = chess.parse_square(value)
                occupied_cells[square] = piece
            
    return occupied_cells

Die Funktion `create_board()` füllt das Schachbrett letztlich mit den Figuren, die in den Dictionaries angegeben wurden. Als
Rückgabewert gibt die Funktion das gefüllte Schachbrett als Objekt der `chess` Library zurück.
Hierzu werden die vorherigen Funktionen verwendet, um die `Piece-Map` zu erstellen, diese 
auf das Board-Objekt angewandt und auf Validität überprüft.
Ist das Board nicht valide, wird ein Fehler ausgegeben.

Ein Problem, welches in Spielsituationen mit dem Turm auftritt, ist die Berechtigung zu einer Rochade.
Damit das Board als valide anerkannt wird, werden die entsprechenden Flags mit der Funktion
`clean_castling_rights` korrekt gesetzt.

> TODO: Was ist mit "korrekt gesetzt" gemeint?
> TODO: Problem entweder vor Funktionsdefinition schreiben oder nach Codezelle, dass Bezug nicht verloren geht

In [ ]:
def create_board():
    local_board = chess.Board()
    occupied_cells = {}
    if check_for_correct_cells():
        occupied_cells |= collect_cells(chess.WHITE, WHITE_POSITIONS.items())
        occupied_cells |= collect_cells(chess.BLACK, BLACK_POSITIONS.items())

        local_board.set_piece_map(occupied_cells)
        local_board.castling_rights = local_board.clean_castling_rights()
        
        # Herausfinden, weshalb momentan angegebenes Board invalid ist
        if not local_board.is_valid():
            display(local_board)
            print("Specified lineup is invalid")
            print("Default board created instead")
            local_board = chess.Board()
    return local_board

## Import der Daten

Für die Bestimmung der Züge der KI werden die $S_n$ Mengen verwendet, die im Notebook ``calculation.ipynb`` berechnet werden.   
Eine Erklärung worum es sich hierbei handelt, findet sich in diesem Notebook.
Die Ergebnisse der Berechnung werden mittels `pickle` serialisiert und in einer ZIP-Datei komprimiert abgespeichert. Zur Verwendung wird das Archiv entpackt und 
die Liste deserialisiert. 
Weitere Informationen zum Inhalt der Datei befinden sich ebenfalls im Notebook ``calculation.ipynb``.   

Die Funktion `load_s_n_sequence(filename)` erhält den Dateinamen (``filename``) als Parameter und gibt die Endspieldaten zurück.

> TODO: Pfadnamen ist doch hier die falsche Beschreibung, oder? Und wenn Pfad, dann sagen, ob absolut oder relativ
> TODO: Rückgabewert beschreiben

In [ ]:
def load_data(filename):
    s_n_sequence_integers = []
    with ZipFile("S_n_Results/" + filename + ".chessAI") as zipped:
        with zipped.open(filename + ".pickle") as calculation:
            tmp = pickle.loads(calculation.read())
            piece_list = tmp[0]
            for item in tmp[1:]:
                s_n_sequence_integers.append(item)
    return piece_list, s_n_sequence_integers

## Den besten Zug für die KI ermitteln
Um mit den berechneten Endspieldaten gegen einen Spieler zu gewinnen, muss jeder Zug, den Weiß macht, optimal sein.
Ein passender Zug für die KI besteht darin die Situation von $S_n$ in einen Zustand zu überführen, in dem sie sich in $S_{n-1}$ befindet.  

Die Funktion `find_next_move(curr_board, s_index, s_n_sequence)` bestimmt für ein übergebenes Board-Objekt einen solchen Spielzug. 
Aus Effizienzgründen wird zusätzlich zum gefundenen Spielzug (`move`) der neue Wert für $n$ zurückgegeben. 

Für diese Berechnung benötigt ``find_next_move`` nachkommende Argumente:
* ``curr_board``: Das Board, für welches der nächste Spielzug berechnet werden soll.
* ``s_index``: Das $n$ eines $S_n$, in welchem sich ``curr_board`` befindet. Wird aus Effizienzgründen übergeben.
* ``s_n_sequence``: Die Liste mit allen $S_n$.

Kann kein Spielzug gefunden werden, gibt die Funktion den Wert -1 zurück.

Die für diesen Ablauf benötigte Funktion `find_situation_in_sequence` wird im Notebook `functions.ipynb` definiert und erklärt.

In [ ]:
def find_next_move(curr_board, s_index, s_n_sequence):
    s_index_new = s_index - 1
    for move in curr_board.legal_moves:
        curr_board.push(move)
        curr_integer = to_integer(curr_board, PIECE_LIST)
        _tmp = find_situation_in_sequence(curr_integer, [s_n_sequence[s_index_new]])
        if _tmp != -1:
            curr_board.pop()
            return s_index_new, move
        curr_board.pop()

    return -1, None

## Globale Variablen für die Anzeige

Nachdem alle Funktionen für die Bestimmung eines Zugs definiert worden sind, gilt es die bereitgestellte UI der ``python-chess`` Bibliothek zu erweitern. 
In Form von globalen Variablen werden UI-Elemente definiert, die für die Eingabe eines neuen Zugs benötigt werden:

- `input_field` = Ein Eingabefeld, in dem der nächste Zug von Weiß eingetragen werden soll.
- `execute_button` = Ein Button, der nach der Auswahl des Zuges diesen auch auf dem Schachbrett ausführt.

In [ ]:
# Sollten diese nicht ebenfalls in caps geschrieben werden?
# TODO: Tooltip anpassen
input_field = widgets.Text()

execute_button = widgets.Button(
    description='Execute Move',
    disabled=False,
    button_style='',
    tooltip='Executes the move selected with the dropdowns "Piece:" and "Move:"',
    icon='check'
)

## Globale Variablen für den aktuellen Spielzustand
Damit der aktuelle Stand des Spiels auch ohne eine lineare Kette von Funktionsaufrufen verfügbar ist, 
werden diese Informationen in globalen Variablen gespeichert.

> TODO: Globale Variablen beschreiben und nennen

In [ ]:
PIECE_LIST, S_N_INTEGERS = load_data(FILE)
S_INDEX = 0
BOARD_INDEX = 0

Weiterhin werden weitere globale Variablen definiert, die einerseits den Dateinamen für die Spielzüge (`filename`), 
andererseits das Spielbrett zum Spielen gegen die KI innehalten (`board`).

> TODO: Filename wurde schon vorher definiert und board wird hier noch gar nicht definiert

Innerhalb der python-chess library werden die Farben des Schachbretts `chess.WHITE` und `chess.BLACK` als boolesche Variablen definiert. 
Deswegen wurde zur späteren Dokumentation, aber auch zum Debuggen eine Funktion geschrieben, 
die die Farbe, die gerade am Zug ist, als String zurückgibt.

In [ ]:
def get_color(turn):
    if turn:
        return "White"
    else:
        return "Black"

Zum Definieren der globalen Variablen `FILENAME` wird die Funktion `update_filename` definiert.

> TODO: Wird FILENAME in diesem Kontext wirklich global benötigt?

In [ ]:
def update_filename(new_value):
    global FILENAME
    FILENAME = new_value

Weiter werden für den Spielbeginn (`reset_board`) und auch für ausgeführte Spielzüge (`reset_input_field`) Funktionen geschrieben, 
die die UI Elemente auf ihren Standardwert zurücksetzen.

> TODO: Reicht diese Beschreibung?

In [ ]:
def reset_input_field():
    global input_field
    input_field.value = ''

In [ ]:
def reset_board():
    global BOARD
    BOARD = create_board()
    global S_INDEX
    global BOARD_INDEX
    integer = to_integer(BOARD, PIECE_LIST)
    S_INDEX = find_situation_in_sequence(integer, S_N_INTEGERS)
    if BOARD.turn:
        S_INDEX, move = find_next_move(BOARD, S_INDEX, S_N_INTEGERS)
        if S_INDEX == -1:
            print("No Move for white found")
        else:
            execute_move(move, BOARD.turn)
            reload_screen()

Zum Durchführen der Spielzüge im Board-Objekt wird die Hilfsfunktion `execute_move` verwendet.
Die Funktion erhält als Parameter einen `chess.Move` und führt diesen auf dem globalen `BOARD` für die Farbe `chess.turn` durch. 
Zusätzlich wird in der Datei, die in unter dem Namen `filename` zu finden ist, für den Zug ein Eintrag hinterlegt.

> TODO: Variablen zu den im Text beschriebenen Parametern mappen

In [ ]:
def execute_move(move, turn):
    global BOARD
    global S_INDEX
    global BOARD_INDEX
    move_file = open("Played_Games/" + FILENAME, "a")
    if turn:
        move_file.write(str(BOARD.fullmove_number) + ". " + move.uci() + " ")
    else:
        move_file.write(move.uci() + "\n")
    move_file.close()
    BOARD.push(move)
    integer = to_integer(BOARD, PIECE_LIST)
    S_INDEX = find_situation_in_sequence(integer, S_N_INTEGERS)

## Weitere Hilfsfunktionen

Das Ziel der Funktion `get_pieces_placed_on_board` besteht darin, von einer mitgegebenen `chess.Color` die Figuren zu bestimmen, die noch auf dem Schachbrett stehen. Zurück gibt sie ein Dictionary, das als `key` die Schachfiguren und als `value` eine Liste mit den Positionen der Figuren besitzt.

> TODO: Überprüfung, ob beide Dictionaries notwendig sind
> TODO: Rückgabewerte genauer beschreiben

In [ ]:
def get_pieces_placed_on_board(color):
    piece_type_to_string = {
        1 : "Pawn",
        2 : "Knight",
        3 : "Bishop",
        4 : "Rook",
        5 : "Queen",
        6 : "King"
    }
    pieces_with_position = {
        "Pawn" : [],
        "Knight" : [],
        "Bishop" : [],
        "Rook" : [],
        "Queen" : [],
        "King" : []
    }
    for piece_square, color_piece in BOARD.piece_map().items():
        if color_piece.color == color:
            pieces_with_position[piece_type_to_string[color_piece.piece_type]].append(chess.square_name(piece_square))
    return pieces_with_position

Bei der Funktion `get_moves_from_square` werden anhand der mitgegebenen `legal_moves` für ein vorgegebenes Feld die Züge selektiert, die man von diesem Feld aus ziehen kann.

> TODO: Parameter werden nicht genügend definiert

In [ ]:
def get_moves_from_square(square, legal_moves):
    moves = []
    for possible_move in legal_moves:
        if square == chess.square_name(possible_move.from_square):
            moves.append(possible_move)
    return moves

Da die Anzeige des UI auf Konsolenausgaben basiert, muss diese auch nach einem neuen Zug geleert werden, 
sodass die Konsole nicht mit den Elementen überflutet wird. Hierfür aktualisiert die Funktion `reload_screen` 
die Ausgabe und zeigt erneut das Schachbrett, die beiden Dropdowns und den Knopf zum Ausführen des Zugs an.

> TODO: Werden keine Dropdowns mehr angezeigt

In [ ]:
def reload_screen():
    clear_output()
    display(BOARD, input_field, execute_button)
    display(Javascript("setTimeout(function focus() {document.querySelector('input').focus()}, 100);"))

Die Funktion `show_end_screen` hingegen zeigt nur das Schachbrett an.

In [ ]:
def show_end_screen():
    clear_output()
    display(BOARD)

Die `game_result(result)` Funktion gibt den Grund, weshalb das Spiel beendet wurde zurück.

> TODO: Gibt nicht den Grund an, hat als Rückgabewert den Grund in String-Form
> TODO: Kann man anstatt der Berechnung mit dem Outcome nicht wieder die Abfragen machen mit is_checkmate() etc.?

In [ ]:
def game_result(result):
    if result == Termination.CHECKMATE:
        return get_color(BOARD.turn) + " has lost because of Checkmate!"
    elif result == Termination.STALEMATE:
        return "It's a draw!"
    elif result == Termination.INSUFFICIENT_MATERIAL:
        return "No side can win the game anymore!"
    elif result == Termination.SEVENTYFIVE_MOVES:
        return "The game is drawn because half-move clock is greater than 150 since a capture or a pwn has been moved."
    elif result == Termination.FIVEFOLD_REPETITION:
        return "The game is drawn because the current position occurred the fifth time!"
    elif result == Termination.FIFTY_MOVES:
        return "The game is drawn because half-move clock is greater than 100 since a capture or a pwn has been moved."
    elif result == Termination.THREEFOLD_REPETITION:
        return "The game is drawn because the current position occurred the third time!"
    elif result == Termination.VARIANT_WIN:
        return get_color(BOARD.turn) + " has won because of variant-specific conditions"
    elif result == Termination.VARIANT_LOSS:
        return get_color(BOARD.turn) + " has lost because of variant-specific conditions"
    elif result == Termination.VARIANT_DRAW:
        return "Game is drawn because of variant-specific conditions!"
    else:
        return "Something went wrong!"

Für den `execute_button` und das `input_field` wird eine Funktion geschrieben, die für das Ausführen des Zuges verantwortlich ist.
Diese wird entweder durch Klicken des Buttons oder Drücken der Enter-Taste aufgerufen. 
Sobald der eingegebene Zug ausgeführt wurde, wird mithilfe der eingelesenen $S_n$ Mengen der Zug für die KI bestimmt. 
Weiterhin wird auch in dieser Funktion überprüft, ob das Spiel bereits beendet wurde.

> TODO: Funktion nochmal kleiner teilen

In [ ]:
def execute_entered_move(change):
    global BOARD
    global S_INDEX
    global BOARD_INDEX
    try:
        if input_field.value != '':
            black_move = chess.Move.from_uci(input_field.value)
            if black_move in list(BOARD.legal_moves):
                execute_move(black_move, BOARD.turn)
                reset_input_field()
                # Next AI move executed by white
                S_INDEX, next_move = find_next_move(BOARD, S_INDEX, S_N_INTEGERS)
                if S_INDEX == -1:
                    print("No Move for white found")
                else:
                    execute_move(next_move, BOARD.turn)
                    reload_screen()
                if BOARD.legal_moves.count() == 0:
                    # If wanted add different endings
                    show_end_screen()
                    result = BOARD.outcome().termination
                    print(game_result(result))
            else:
                print("Entered a wrong move. Please try again!")
                print(input_field.value)
                time.sleep(2)
                reload_screen()
        else:
            print("Enter a move!")
            time.sleep(2)
            reload_screen()
    except ValueError:
        print("Entered a wrong move. Please try again!")
        print(input_field.value)
        time.sleep(2)
        reload_screen()

Damit die Funktion beim Klicken des `execute_button` oder durch Drücken der Enter-Taste ausgeführt wird, muss diese den `on_click` Events zugewiesen werden.

In [ ]:
execute_button.on_click(execute_entered_move)
input_field.on_submit(execute_entered_move)

Die Funktion `start_game` bereitet alle Parameter für einen Spielverlauf vor. Dazu gehört:
- Erstellung eines neuen Boards.
- Das Erstellen einer neuen Historie für das neu begonnene Spiel.
- Anzeigen der neuen Spielsituation.

In [ ]:
def start_game():
    global BOARD
    BOARD = create_board()
    update_filename("Move-History_" + str(datetime.today().replace(microsecond=0)).replace(":","_") + ".txt")
    move_file = open("Played_Games/" + FILENAME, "a")
    move_file.write(BOARD.fen() + "\n")
    move_file.close()
    reset_board()

Mit dem Aufruf der Funktion `start_game` kann nun ein Spiel gegen die KI gestartet werden.

In [ ]:
start_game()
